In [1]:
from datasets import load_dataset, Audio

SAMPLING_RATE=24_000
# Load the LJ Speech dataset
dataset = load_dataset("MikhailT/lj-speech")
dataset = dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
dataset = dataset.with_format("torch")
len(dataset["full"])

/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


13100

In [2]:
from data_pipeline.utils.codec import MimiCodec

mimi_model = MimiCodec()

/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/transformers/models/mimi/modeling_mimi.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [3]:
first_items = dataset["full"][0:16]
audios = [row["array"] for row in first_items["audio"]]
wavs = mimi_model.encode_batch(audios)

In [4]:
[l.shape for l in wavs]

[torch.Size([8, 120]),
 torch.Size([8, 23]),
 torch.Size([8, 120]),
 torch.Size([8, 64]),
 torch.Size([8, 101]),
 torch.Size([8, 71]),
 torch.Size([8, 104]),
 torch.Size([8, 22]),
 torch.Size([8, 94]),
 torch.Size([8, 110]),
 torch.Size([8, 56]),
 torch.Size([8, 102]),
 torch.Size([8, 32]),
 torch.Size([8, 124]),
 torch.Size([8, 115]),
 torch.Size([8, 65])]

In [5]:
from IPython import display

out_pcm = mimi_model.decode(wavs[15])

# Convert to CPU and get numpy array for playback
audio_data = out_pcm.numpy()

# Create and display audio widget
# Note: sample_rate=24000 matches your original save command
display.display(display.Audio(audio_data, rate=24000, autoplay=False))

In [6]:
def batch_wav_encoder(batch):
    audios = [audio["array"] for audio in batch["audio"]]
    return {
        "codes": mimi_model.encode_batch(audios)
    }

dataset = dataset.map(batch_wav_encoder, batched=True, batch_size=24, remove_columns=["audio"])

Map: 100%|██████████| 13100/13100 [01:44<00:00, 125.71 examples/s]


In [11]:
dataset.column_names

{'full': ['file', 'spoken_text', 'normalized_text', 'codes']}

In [14]:
dataset = dataset.rename_column(original_column_name="normalized_text", new_column_name="text_normalized")
dataset.save_to_disk("../../datasets/encoded_ljspeech")

Saving the dataset (1/1 shards): 100%|██████████| 13100/13100 [00:00<00:00, 70390.19 examples/s]
